## Задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

from datetime import datetime, timedelta

In [36]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'

In [37]:
post_times = []
post_titles = []
post_links = []
post_body = []
for keyword in KEYWORDS:
    params = {
        'q':keyword
    }
    req = requests.get(URL, params)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_times.append(post.find('span', class_='post__time').text)
        post_titles.append(post.find('h2', class_='post__title').text)
        post_links.append(post.find('a', class_='btn').get('href'))
        req2 = requests.get(post.find('a', class_='btn').get('href'))
        content = BeautifulSoup(req2.text, 'html.parser')
        
post_titles = list(map(lambda x: x.strip(), post_titles))
articles = pd.DataFrame({'время':post_times, 'заголовок':post_titles, 'ссылка':post_links})
articles.head (10)

,время,заголовок,ссылка
0,сегодня в 17:18,Создание современных процессов CI/CD для бессе...,https://habr.com/ru/company/southbridge/blog/5...
1,сегодня в 17:11,Подводя итоги 2020 года,https://habr.com/ru/post/536090/#habracut
2,сегодня в 16:56,Открытые материалы: курс по вычислительной ней...,https://habr.com/ru/company/JetBrains-educatio...
3,сегодня в 15:30,Как дата-саентисты в ВК делают рекламу эффекти...,https://habr.com/ru/company/ruvds/blog/536072/...
4,сегодня в 14:55,А мог побороться с CoD… История создания шутер...,https://habr.com/ru/post/535972/#habracut
5,сегодня в 14:53,lsFusion vis 1С,https://habr.com/ru/post/536080/#habracut
6,сегодня в 14:23,Нейротипология и нейромаркетинг будущего,https://habr.com/ru/post/536076/#habracut
7,сегодня в 14:14,Хабрастыд-2020,https://habr.com/ru/post/509418/#habracut
8,сегодня в 14:12,Швеция: что ждать от релокейта к суровым север...,https://habr.com/ru/company/gms/blog/535822/#h...
9,сегодня в 14:04,Хакатоны Зима 2020,https://habr.com/ru/post/536068/#habracut


## Задание 2.

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [159]:
URL   = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['reddeade@gmail.com']

headers = {'Accept':'application/json',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Connection':'keep-alive',
'Content-Length':'32',
'Content-Type':'application/json;charset=UTF-8',
'Host':'identityprotection.avast.com',
'Origin':'https://www.avast.com',
'Referer':'https://www.avast.com/',
'Sec-Fetch-Dest':'empty',
'Sec-Fetch-Mode':'cors',
'Sec-Fetch-Site':'same-site',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
'Vaar-Header-App-Product':'hackcheck-web-avast',
'Vaar-Version':'0'}

#POST-orders
params = {'emailAddresses': EMAIL}

# POST-orders
req = requests.post(URL, json = params, headers = headers)
if (req.status_code == 200) & (len(req.text)) > 0:
    dict_ = json.loads(req.text)
    rezult = []

    df_hack_columns = ('mail', 'dt', 'source', 'descript')
    df_hack = pd.DataFrame(columns = df_hack_columns)

#Dataframe formation
    for item_mail in EMAIL:
        for item in dict_['summary'][item_mail]['breaches']:
            row_ = [item_mail,\
                    dict_['breaches'][str(item)]['publishDate'],\
                    dict_['breaches'][str(item)]['site'],\
                    dict_['breaches'][str(item)]['description']]
            df_hack = df_hack.append( pd.DataFrame([row_], columns = df_hack_columns), ignore_index = True )
    print(df_hack)
else:
    print('НЕ ПУСТИЛ checkbox')

НЕ ПУСТИЛ checkbox
